In [1]:
import pandas as pd
import numpy as np

In [3]:
# # effect size tables # #
#FDR - 0.05
mcll=pd.read_csv("/home/ls/rachelcw/projects/LEAFCUTTER/DS/DS.five_percent/analysis.20230115/fdr0.05/filtered.a4.20230115_effect_sizes.txt",sep=' ')
ucll=pd.read_csv("/home/ls/rachelcw/projects/LEAFCUTTER/DS/DS.five_percent/analysis.20230115/fdr0.05/filtered.a6.20230115_effect_sizes.txt",sep=' ')
mucll=pd.read_csv("/home/ls/rachelcw/projects/LEAFCUTTER/DS/DS.five_percent/analysis.20230115/fdr0.05/filtered.a2.20230115_effect_sizes.txt",sep=' ')

In [4]:
# # junction column, cll column # #
table = pd.merge(mcll[['intron','unmut','mut','deltapsi']],ucll[['intron','unmut','mut','deltapsi']], how='outer')
table=pd.merge(table, mucll[['intron','unmut','mut','deltapsi']], how='outer')
table['analysis_set'] = None
table.loc[table['intron'].isin(mcll['intron']), 'analysis_set'] = 'm'
table.loc[table['intron'].isin(ucll['intron']), 'analysis_set'] = 'u'
table.loc[table['intron'].isin(mucll['intron']),'analysis_set' ] = 'mu'
#table.rename({'intron':'junction'},inplace=True)
table[['junction','cluster']]=table['intron'].str.rsplit(':',expand=True,n=1)
# table.drop(columns='intron',inplace=True)
# table = table.reindex(columns=['junction','analysis_set','cluster'])

In [6]:
# # cluster significance # # 
#FDR - 0.05
mcll=pd.read_csv("/home/ls/rachelcw/projects/LEAFCUTTER/DS/DS.five_percent/analysis.20230115/fdr0.05/filtered.a4.20230115_cluster_significance.txt",sep=' ')
ucll=pd.read_csv("/home/ls/rachelcw/projects/LEAFCUTTER/DS/DS.five_percent/analysis.20230115/fdr0.05/filtered.a6.20230115_cluster_significance.txt",sep=' ')
mucll=pd.read_csv("/home/ls/rachelcw/projects/LEAFCUTTER/DS/DS.five_percent/analysis.20230115/fdr0.05/filtered.a2.20230115_cluster_significance.txt",sep=' ')

In [7]:

cluster_table=pd.merge(mcll[['cluster','p.adjust']],ucll[['cluster','p.adjust']],on='cluster',how='outer',suffixes=['_m','_u'])
cluster_table=cluster_table.merge(mucll[['cluster','p.adjust']],on='cluster',how='outer',suffixes=['_mu'])
cluster_table['cluster']=cluster_table['cluster'].str.rsplit(':',expand=True,n=1)[1]

In [8]:
table=table.merge(cluster_table,how='left',on='cluster')

In [9]:
# gene name
import pyensembl
table['gene_name']=pd.NaT
# table['identified']=pd.NaT
data = pyensembl.Genome(
    reference_name='GRCh37',
    annotation_name='my_genome_lab',
    gtf_path_or_url='/private1/private/resources/gencode19_noChrPrefix_mitoMT.gtf')
data.index()
for i,j in enumerate(table.junction):
    j=j.split(':')
    chr=j[0].replace('chr','')
    start=int(j[1])
    end=int(j[2])
    gene_name = data.gene_names_at_locus(contig=chr, position=start,end=end)
    transcript=data.transcript_ids_at_locus(contig=chr,position=start,end=end)
    if gene_name != []:
        table.at[i,'gene_name']=gene_name

table.dropna(subset=['gene_name'], inplace=True)


In [10]:
# filter out when unmut bigger than mut (deltapsi < 1) #
table.rename(columns={'p.adjust':'p.adjust_mu'},inplace=True)
table = table[table.deltapsi > 0] # mut-unmut >1

In [14]:
# the junction (event) is identified in GTF or not #
fiveprime=pd.read_csv("/home/ls/rachelcw/projects/BIO/annontation_code.20221225_fiveprime.bed.gz", sep='\t',compression='gzip', header=None,usecols=[0,2,5] ,names=['chrom', 'start_int',  'strand'])
threeprime=pd.read_csv("/home/ls/rachelcw/projects/BIO/annontation_code.20221225_threeprime.bed.gz", sep='\t',compression='gzip', header=None, usecols=[0,2,5],names=['chrom', 'start_int', 'strand'])
threeprime.head()
for junc in table.junction:
    junc=junc.split(':')
    chr=junc[0].replace('chr','')
    start=int(junc[1])
    end=int(junc[2])

/home/ls/rachelcw/miniconda3/envs/bio/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,chrom,start_int,strand
0,X,99891606,-
1,X,99890556,-
2,X,99890176,-
3,X,99888929,-
4,X,99888403,-


In [9]:
# essential gene to sf3b1 #
gene_dep=pd.read_csv("/home/ls/rachelcw/projects/CRISPR_gene_dependency.csv")

In [14]:
start=115890428
end=115890906
exons=data.exons_at_locus(contig='10', position=start,end=end)
print(exons)
for exon in exons:
    if exon.start<end:
        print('alt 3 ss')
        print(exon)
    elif exon.end>start:
        print('alt 5 ss')
        print(exon)
    

[Exon(exon_id='ENSE00001839081.1', gene_id='ENSG00000165813.12', gene_name='C10orf118', contig='10', start=115890785, end=115891095, strand='-'), Exon(exon_id='ENSE00001873682.1', gene_id='ENSG00000165813.12', gene_name='C10orf118', contig='10', start=115889958, end=115890428, strand='-'), Exon(exon_id='ENSE00003514147.1', gene_id='ENSG00000165813.12', gene_name='C10orf118', contig='10', start=115890906, end=115891095, strand='-'), Exon(exon_id='ENSE00003676580.1', gene_id='ENSG00000165813.12', gene_name='C10orf118', contig='10', start=115890906, end=115891095, strand='-')]
alt 3 ss
Exon(exon_id='ENSE00001839081.1', gene_id='ENSG00000165813.12', gene_name='C10orf118', contig='10', start=115890785, end=115891095, strand='-')
alt 3 ss
Exon(exon_id='ENSE00001873682.1', gene_id='ENSG00000165813.12', gene_name='C10orf118', contig='10', start=115889958, end=115890428, strand='-')
alt 5 ss
Exon(exon_id='ENSE00003514147.1', gene_id='ENSG00000165813.12', gene_name='C10orf118', contig='10', star

In [ ]:
# # alternative 5/3 prime splice site # #
for j in table.junction:
    j=j.split(':')
    chr=j[0].replace('chr','')
    start=int(j[1])
    end=int(j[2])
    # Find the intron with the desired junction coordinates
    
    for transcript in data.transcripts(chr):
    # Iterate over the exons of the gene
        #exons=data.exon_ids_of_gene_id(gene.gene_id)
        for exon in transcript.exons:
            

            # Check if the exon has an alternative 5' splice site
            # if exon.start < end + 1:
            #     print(f'Exon {exon.id} has an alternative 5\' splice site')
            # # Check if the exon has an alternative 3' splice site
            # if exon.end == start - 1:
            #     print(f'Exon {exon.id} has an alternative 3\' splice site')
        break